In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
# Set the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:

#### GOING TO IMPLEMENT THIS #####
import sys
import os
import math
import random
from itertools import product
import numpy
import torch
import torchvision
import torchvision.transforms as transforms

sys.path.append(os.path.abspath("/content/drive/MyDrive/UKP/"))
from distance_functions_final import *

# Set the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lmbda_range = np.power(10.0, range(-7, 2))
lmbda_range = np.concatenate((lmbda_range, [0]))
sigma_range = np.power(10.0, range(-3, 3))


def evaluate_distances(A, B, i, j):

    all_dists = {}

    try:
        evals_a, evecs_a = np.linalg.eigh(A @ A.T)
        evals_b, evecs_b = np.linalg.eigh(B @ B.T)

        _, s, _, transformed_a, transformed_b = cca_decomp(A, B, evals_a, evecs_a, evals_b, evecs_b)

        all_dists['mean_sq_cca_e2e'] = mean_sq_cca_corr(s)
        all_dists['mean_cca_e2e'] = mean_cca_corr(s)
        all_dists['pwcca_dist_e2e'] = pwcca_dist(A, s, transformed_a)

        all_dists['lin_cka_dist'] = lin_cka_dist(A, B)
        all_dists['lin_cka_prime_dist'] = lin_cka_prime_dist(A, B)

        all_dists['procrustes'] = procrustes(A, B)

        for lmbda in lmbda_range:
            all_dists[f'GULP_dist_{lmbda:e}'] = GULP_dist(A, B, evals_a, evecs_a, evals_b, evecs_b, lmbda=lmbda)
            print(f'Lambda = {lmbda:e} results being computed for GULP for pair index ({i},{j})', flush=True)

        A = torch.tensor(A, dtype=torch.float32).to(device)
        B = torch.tensor(B, dtype=torch.float32).to(device)

        first_lambda_bool = True

        for lmbda in lmbda_range:
            for k in range(len(sigma_range)):
                sigma = sigma_range[k]
                print(f'Lambda = {lmbda:e} and sigma = {sigma:e} results being computed for UKP and CKA for pair index ({i},{j})', flush=True)
                A_kernelized = cuda_rbf_kernel(A.T, sigma=sigma).to(device)
                B_kernelized = cuda_rbf_kernel(B.T, sigma=sigma).to(device)

                try:
                    evals_a_K, evecs_a_K = torch.linalg.eigh(A_kernelized)
                except torch._C._LinAlgError as e:
                    evals_a_K = None
                    evecs_a_K = None

                try:
                    evals_b_K, evecs_b_K = torch.linalg.eigh(B_kernelized)
                except torch._C._LinAlgError as e:
                    evals_b_K = None
                    evecs_b_K = None

                all_dists[f'UKP_dist_RBF_{sigma:e}_{lmbda:e}'] = UKP_dist(A, B, evals_a_K, evecs_a_K, evals_b_K,
                                                                          evecs_b_K, A_kernelized, B_kernelized,
                                                                          kerneltype=1, sigma=sigma, lmbda=lmbda)
                if first_lambda_bool == True:
                    all_dists[f'CKA_dist_RBF_{sigma:e}'] = cka_dist(A, B, A_kernelized, B_kernelized, kerneltype=1,
                                                                    sigma=sigma)

                del A_kernelized
                del B_kernelized
                del evals_a_K
                del evecs_a_K
                del evals_b_K
                del evecs_b_K

                A_kernelized = cuda_laplacian_kernel(A.T, sigma=sigma).to(device)
                B_kernelized = cuda_laplacian_kernel(B.T, sigma=sigma).to(device)

                try:
                    evals_a_K, evecs_a_K = torch.linalg.eigh(A_kernelized)
                except torch._C._LinAlgError as e:
                    evals_a_K = None
                    evecs_a_K = None

                try:
                    evals_b_K, evecs_b_K = torch.linalg.eigh(B_kernelized)
                except torch._C._LinAlgError as e:
                    evals_b_K = None
                    evecs_b_K = None

                all_dists[f'UKP_dist_Laplace_{sigma}_{lmbda}'] = UKP_dist(A, B, evals_a_K, evecs_a_K, evals_b_K,
                                                                          evecs_b_K, A_kernelized, B_kernelized,
                                                                          kerneltype=2, sigma=sigma, lmbda=lmbda)
                if first_lambda_bool == True:
                    all_dists[f'CKA_dist_Laplace_{sigma}'] = cka_dist(A, B, A_kernelized, B_kernelized, kerneltype=2,
                                                                      sigma=sigma)

            first_lambda_bool = False

        del A_kernelized
        del B_kernelized
        del evals_a_K
        del evecs_a_K
        del evals_b_K
        del evecs_b_K

    except numpy._core._exceptions._ArrayMemoryError as e:
        print(f'The following error occured so returning None: {e}')

        all_dists['mean_sq_cca_e2e'] = None
        all_dists['mean_cca_e2e'] = None
        all_dists['pwcca_dist_e2e'] = None

        all_dists['lin_cka_dist'] = None
        all_dists['lin_cka_prime_dist'] = None

        all_dists['procrustes'] = None

        for lmbda in lmbda_range:
            all_dists[f'GULP_dist_{lmbda:e}'] = None

        first_lambda_bool = True

        for lmbda in lmbda_range:
            for k in range(len(sigma_range)):
                sigma = sigma_range[k]

                all_dists[f'UKP_dist_RBF_{sigma:e}_{lmbda:e}'] = None
                if first_lambda_bool == True:
                    all_dists[f'CKA_dist_RBF_{sigma:e}'] = None

                all_dists[f'UKP_dist_Laplace_{sigma}_{lmbda}'] = None
                if first_lambda_bool == True:
                    all_dists[f'CKA_dist_Laplace_{sigma}'] = None

            first_lambda_bool = False

    return all_dists

subset = "train"
mode = "eval"
n = 20000
dist_n = 5000
reps = []
reps_folder = f"/content/drive/MyDrive/UKP/mnist_experiments/reps/{subset}/{n}_{mode}"
filenames = os.listdir(reps_folder)
for filename in filenames:
  if "saved" not in filename and "seed2" in filename and "depth3" not in filename and "depth6" not in filename and "depth8":
    reps.append(filename[:-4])


reps = np.sort(reps)

model_names = reps

folder = f"/content/drive/MyDrive/UKP/mnist_experiments/distances/widthdepth/{dist_n}_{subset}/"
total_models = len(model_names)

dist_pairs_saved = np.zeros((total_models, total_models), dtype=bool)
if os.path.exists(f"{folder}/stats.npz"):
    dist_pairs_saved = np.load(f"{folder}/stats.npz")["dist_pairs_saved"]
    print(f"{np.sum(dist_pairs_saved)} existing pairs", flush=True)

model_pairs = []
model_pairs_index = []
for i in range(total_models):
    for j in range(i + 1, total_models):
        if not dist_pairs_saved[i, j]:
            model_pairs.append((model_names[i], model_names[j]))
            model_pairs_index.append((i, j))
# random.Random(4).shuffle(model_pairs)

total_pairs = len(model_pairs)

batch_pairs = model_pairs
print(len(batch_pairs), flush=True)







In [ ]:
for i in range(total_models):
    for j in range(i + 1, total_models):
        if dist_pairs_saved[i,j] == False:
            name1 = model_names[i]
            name2 = model_names[j]

            print(f'Computing {name1}, {name2}', flush=True)

            rep1 = np.load(f"{reps_folder}/{name1}.npy")
            rep1 = rep1[:, :dist_n]
            # center and normalize
            rep1 = rep1 - rep1.mean(axis=1, keepdims=True)
            rep1 = math.sqrt(dist_n) * rep1 / np.linalg.norm(rep1)

            rep2 = np.load(f"{reps_folder}/{name2}.npy")
            rep2 = rep2[:, :dist_n]
            # center and normalize
            rep2 = rep2 - rep2.mean(axis=1, keepdims=True)
            rep2 = math.sqrt(dist_n) * rep2 / np.linalg.norm(rep2)

            all_dists = evaluate_distances(rep1, rep2, i, j)
            np.savez(f"{folder}{model_names[i]}_{model_names[j]}.npz", **all_dists)
            for dist_name in all_dists:
                dist = all_dists[dist_name]
                print(f'{dist_name}: {dist}', flush=True)
            print(flush=True)

            dist_pairs_saved[i, j] = True
            np.savez(f"{folder}stats.npz", model_names=model_names, dist_pairs_saved=dist_pairs_saved)

            del rep1
            del rep2
            del all_dists

            # Ensure all GPU operations are done
            torch.cuda.synchronize()

            # Clear GPU memory cache
            torch.cuda.empty_cache()

            # Optionally, print memory status (for debugging)
            print(torch.cuda.memory_summary())

        else:
            print(f'Computation of {i}={model_names[i]},{j}={model_names[i]} already done !', flush=True)
